In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [164]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [165]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {}
for word in world_set:
    new_id = len(vocab)
    vocab[word] = new_id

In [166]:
print(vocab)

{'p': 0, 'd': 1, 'w': 2, 'g': 3, 'h': 4, 'n': 5, 'o': 6, '.': 7, 'y': 8, 'a': 9, "'": 10, 'f': 11, 'e': 12, ' ': 13, 'B': 14, 'i': 15, 's': 16, 'l': 17, 'b': 18, ',': 19, 'r': 20, 'u': 21, 'c': 22, 'm': 23, 't': 24, 'k': 25}


In [167]:
print(vocab)

{'p': 0, 'd': 1, 'w': 2, 'g': 3, 'h': 4, 'n': 5, 'o': 6, '.': 7, 'y': 8, 'a': 9, "'": 10, 'f': 11, 'e': 12, ' ': 13, 'B': 14, 'i': 15, 's': 16, 'l': 17, 'b': 18, ',': 19, 'r': 20, 'u': 21, 'c': 22, 'm': 23, 't': 24, 'k': 25}


In [168]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [169]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [202]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+10]
  y_str = sentence[i+1:i+11]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [197]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[14, 20, 15, 22, 25, 13, 2, 9, 17, 17]
[20, 15, 22, 25, 13, 2, 9, 17, 17, 16]


In [206]:
x_data

[[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0.,

In [203]:
for i in range(len(x_data)):
    for j in range(sequence_length):
        x_data[i][j] = np.eye(1,vocab_size,k=x_data[i][j])

In [204]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = x_data 

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
X = X.reshape(188,10,26)

In [210]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [211]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [212]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([20, 15, 22, 25, 13,  2,  9, 17, 17, 16])


In [213]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim,num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim,vocab_size)
    
  def forward(self, x):
    x, state = self.rnn(x)
    x = self.fc(x.reshape((-1,sequence_length,x.shape[-1])))
    
    return x

In [214]:
net = Net(vocab_size, hidden_size, 2)

In [215]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [216]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [217]:
outputs.view(-1,outputs.shape[-1]).shape, Y.view(-1).shape

(torch.Size([1880, 26]), torch.Size([1880]))

In [218]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, outputs.shape[-1]), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggcgggggggggggggggg
g                        t   g    t     t                              t                        g            t                                  t                                                    
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

rick walls are there tor a reaton and you must not thenk that the brick walls are 't there to te p us outh but retherein this way that the brick walls are there to thow us how badly we want thinkth
rick walls are there tor a reaton and you must not thenk that the brick walls are 't there to te p us outh but retherein this way that the brick walls are there to thow us how badly we want thinkth
rick walls are there tor a reaton and you must not think that the brick walls are 't there to te p us out, but rether in this way that the brick walls are there to thow us how badly we want thinkt,
rick walls are there tor a reaton and you must not think that the brick walls are 't there to te p us out, but rether in this way that the brick walls are there to thow us how badly we want thinkt,
rick walls are there tor a reaton and you must not think that the brick walls are 't there to teep us out, but rather in this way that the brick walls are there to thow us how badly we want thinkt,
rick walls

In [219]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinks,"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [9]:
sentence = ("푸틴 대통령은 우크라이나 침공과 관련해서는"
           "'특별 군사작전' 중인 우리 군이 돈바스 전사들과 함께 러시아를 위해,"
           "도네츠크·루한스크 인민공화국 국민의 평화로운 삶을 위해 싸우고 있다고 말했다.")

In [5]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {}
for word in world_set:
    new_id = len(vocab)
    vocab[word] = new_id

In [6]:
print(vocab)

{'스': 0, '작': 1, '민': 2, '라': 3, '께': 4, '공': 5, '를': 6, '이': 7, ' ': 8, '함': 9, '특': 10, '국': 11, '로': 12, '틴': 13, '관': 14, '싸': 15, '별': 16, '대': 17, '령': 18, '츠': 19, '루': 20, '을': 21, '서': 22, '도': 23, '시': 24, '는': 25, '평': 26, '전': 27, '운': 28, ',': 29, '한': 30, '크': 31, '돈': 32, '사': 33, '은': 34, '중': 35, '과': 36, '.': 37, '들': 38, '인': 39, '아': 40, '·': 41, '고': 42, '네': 43, '있': 44, '의': 45, '말': 46, '련': 47, '해': 48, '위': 49, '리': 50, '러': 51, '다': 52, '통': 53, '나': 54, '화': 55, "'": 56, '삶': 57, '바': 58, '했': 59, '군': 60, '푸': 61, '침': 62, '우': 63}


In [7]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 64


In [17]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [9]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+10]
  y_str = sentence[i+1:i+11]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 푸틴 대통령은 우크 -> 틴 대통령은 우크라
1 틴 대통령은 우크라 ->  대통령은 우크라이
2  대통령은 우크라이 -> 대통령은 우크라이나
3 대통령은 우크라이나 -> 통령은 우크라이나 
4 통령은 우크라이나  -> 령은 우크라이나 침
5 령은 우크라이나 침 -> 은 우크라이나 침공
6 은 우크라이나 침공 ->  우크라이나 침공과
7  우크라이나 침공과 -> 우크라이나 침공과 
8 우크라이나 침공과  -> 크라이나 침공과 관
9 크라이나 침공과 관 -> 라이나 침공과 관련
10 라이나 침공과 관련 -> 이나 침공과 관련해
11 이나 침공과 관련해 -> 나 침공과 관련해서
12 나 침공과 관련해서 ->  침공과 관련해서는
13  침공과 관련해서는 -> 침공과 관련해서는'
14 침공과 관련해서는' -> 공과 관련해서는'특
15 공과 관련해서는'특 -> 과 관련해서는'특별
16 과 관련해서는'특별 ->  관련해서는'특별 
17  관련해서는'특별  -> 관련해서는'특별 군
18 관련해서는'특별 군 -> 련해서는'특별 군사
19 련해서는'특별 군사 -> 해서는'특별 군사작
20 해서는'특별 군사작 -> 서는'특별 군사작전
21 서는'특별 군사작전 -> 는'특별 군사작전'
22 는'특별 군사작전' -> '특별 군사작전' 
23 '특별 군사작전'  -> 특별 군사작전' 중
24 특별 군사작전' 중 -> 별 군사작전' 중인
25 별 군사작전' 중인 ->  군사작전' 중인 
26  군사작전' 중인  -> 군사작전' 중인 우
27 군사작전' 중인 우 -> 사작전' 중인 우리
28 사작전' 중인 우리 -> 작전' 중인 우리 
29 작전' 중인 우리  -> 전' 중인 우리 군
30 전' 중인 우리 군 -> ' 중인 우리 군이
31 ' 중인 우리 군이 ->  중인 우리 군이 
32  중인 우리 군이  -> 중인 우리 군이 돈
33 중인 우리 군이 돈 -> 인 우리 군이 돈바
34 인 우리 군이 돈바 ->  우리 군이 돈바스
35  우리 군이 돈바스 -> 우리 군이 돈바스 
36

In [10]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[61, 13, 8, 17, 53, 18, 34, 8, 63, 31]
[13, 8, 17, 53, 18, 34, 8, 63, 31, 3]


In [13]:
for i in range(len(x_data)):
    for j in range(sequence_length):
        x_data[i][j] = np.eye(1,vocab_size,k=x_data[i][j])

In [16]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = x_data 

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
X = X.reshape(95,10,64)

In [17]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([95, 10, 64])
레이블의 크기 : torch.Size([95, 10])


In [31]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim,num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim,vocab_size)
    
  def forward(self, x):
    x, state = self.rnn(x)
    x = self.fc(x.reshape((-1,sequence_length,x.shape[-1])))
    
    return x

In [19]:
net = Net(vocab_size, hidden_size, 2)
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print(outputs.shape)

torch.Size([95, 10, 64])


In [20]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, outputs.shape[-1]), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

돈돈돈돈돈돈돈돈돈돈돈돈를돈돈돈돈를돈를돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈를돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈돈를를돈돈돈돈돈돈돈돈돈를돈돈를돈돈돈돈돈돈돈돈돈돈돈돈돈를돈돈돈돈돈돈돈돈돈
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                           해       스스   해                               
   해   우크     스                                            해       한스  우해                 해             
  우민   우크    우스   민해       사              스스  사       해

In [21]:
predict_str

"틴 대통령은 우크라이나 침공과 관련해서는'특별 군사작전' 중인 우리 군이 돈바스 전사들과 함께 러시아를 위해,도네츠크·루한스크 인민공화국 국민의 평화로운 삶을 위해 싸우고 있다고 말했다."

In [3]:
! pip install konlpy

In [6]:
from konlpy.tag import Komoran

In [10]:
tokenizer= Komoran()  
print(tokenizer.morphs(sentence))
print(tokenizer.pos(sentence))  
print(tokenizer.nouns(sentence))  

['푸틴', '대통령', '은', '우크라이나', '침공', '과', '관련', '하', '아서', '는', "'", '특별', '군사', '작전', "'", '중', '이', 'ㄴ', '우리', '군', '이', '돈바스', '전사', '들', '과', '함께', '러시아', '를', '위하', '아', ',', '도네츠크', '·', '루한스크', '인민공화국', '국민', '의', '평화', '롭', 'ㄴ', '삶', '을', '위하', '아', '싸우', '고', '있', '다고', '말', '하', '았', '다', '.']
[('푸틴', 'NNP'), ('대통령', 'NNG'), ('은', 'JX'), ('우크라이나', 'NNP'), ('침공', 'NNP'), ('과', 'JC'), ('관련', 'NNG'), ('하', 'XSV'), ('아서', 'EC'), ('는', 'JX'), ("'", 'SS'), ('특별', 'XR'), ('군사', 'NNP'), ('작전', 'NNP'), ("'", 'SS'), ('중', 'NNB'), ('이', 'VCP'), ('ㄴ', 'ETM'), ('우리', 'NP'), ('군', 'NNG'), ('이', 'JKS'), ('돈바스', 'NNP'), ('전사', 'NNG'), ('들', 'XSN'), ('과', 'JC'), ('함께', 'MAG'), ('러시아', 'NNP'), ('를', 'JKO'), ('위하', 'VV'), ('아', 'EC'), (',', 'SP'), ('도네츠크', 'NNP'), ('·', 'SP'), ('루한스크', 'NNP'), ('인민공화국', 'NNP'), ('국민', 'NNG'), ('의', 'JKG'), ('평화', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETM'), ('삶', 'NNG'), ('을', 'JKO'), ('위하', 'VV'), ('아', 'EC'), ('싸우', 'VV'), ('고', 'EC'), ('있', 'VV'), ('다고', 'EC'), ('말', 'N

In [14]:
world_set=tokenizer.morphs(sentence)
vocab = {}
for word in world_set:
    new_id = len(vocab)
    vocab[word] = new_id

In [19]:
vocab

{'푸틴': 0,
 '대통령': 1,
 '은': 2,
 '우크라이나': 3,
 '침공': 4,
 '과': 22,
 '관련': 6,
 '하': 43,
 '아서': 8,
 '는': 9,
 "'": 14,
 '특별': 11,
 '군사': 12,
 '작전': 13,
 '중': 14,
 '이': 19,
 'ㄴ': 36,
 '우리': 17,
 '군': 18,
 '돈바스': 19,
 '전사': 20,
 '들': 21,
 '함께': 22,
 '러시아': 23,
 '를': 24,
 '위하': 38,
 '아': 38,
 ',': 27,
 '도네츠크': 28,
 '·': 29,
 '루한스크': 30,
 '인민공화국': 31,
 '국민': 32,
 '의': 33,
 '평화': 34,
 '롭': 35,
 '삶': 36,
 '을': 37,
 '싸우': 38,
 '고': 39,
 '있': 40,
 '다고': 41,
 '말': 42,
 '았': 43,
 '다': 44,
 '.': 45}

In [15]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 46


In [22]:
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(world_set) - sequence_length):
  x_str = world_set[i:i+10]
  y_str = world_set[i+1:i+11]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 ['푸틴', '대통령', '은', '우크라이나', '침공', '과', '관련', '하', '아서', '는'] -> ['대통령', '은', '우크라이나', '침공', '과', '관련', '하', '아서', '는', "'"]
1 ['대통령', '은', '우크라이나', '침공', '과', '관련', '하', '아서', '는', "'"] -> ['은', '우크라이나', '침공', '과', '관련', '하', '아서', '는', "'", '특별']
2 ['은', '우크라이나', '침공', '과', '관련', '하', '아서', '는', "'", '특별'] -> ['우크라이나', '침공', '과', '관련', '하', '아서', '는', "'", '특별', '군사']
3 ['우크라이나', '침공', '과', '관련', '하', '아서', '는', "'", '특별', '군사'] -> ['침공', '과', '관련', '하', '아서', '는', "'", '특별', '군사', '작전']
4 ['침공', '과', '관련', '하', '아서', '는', "'", '특별', '군사', '작전'] -> ['과', '관련', '하', '아서', '는', "'", '특별', '군사', '작전', "'"]
5 ['과', '관련', '하', '아서', '는', "'", '특별', '군사', '작전', "'"] -> ['관련', '하', '아서', '는', "'", '특별', '군사', '작전', "'", '중']
6 ['관련', '하', '아서', '는', "'", '특별', '군사', '작전', "'", '중'] -> ['하', '아서', '는', "'", '특별', '군사', '작전', "'", '중', '이']
7 ['하', '아서', '는', "'", '특별', '군사', '작전', "'", '중', '이'] -> ['아서', '는', "'", '특별', '군사', '작전', "'", '중', '이', 'ㄴ']
8 ['아서', '는', "'", '특별', '군사', '작전', "

In [23]:
print(x_data[0])
print(y_data[0])

[0, 1, 2, 3, 4, 22, 6, 43, 8, 9]
[1, 2, 3, 4, 22, 6, 43, 8, 9, 14]


In [26]:
for i in range(len(x_data)):
    for j in range(sequence_length):
        x_data[i][j] = np.eye(1,vocab_size,k=x_data[i][j])

In [29]:
x_one_hot = x_data 

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
X = X.reshape(43,10,46)

In [30]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([43, 10, 46])
레이블의 크기 : torch.Size([43, 10])


In [32]:
net = Net(vocab_size, hidden_size, 2)
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print(outputs.shape)

torch.Size([43, 10, 46])


In [33]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, outputs.shape[-1]), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

,삶우리들삶삶삶들삶삶삶하들삶삶삶우리··삶삶··'삶·들삶들들삶삶우리'삶들'삶삶우리·삶들들들삶삶삶삶삶들'
·····ㄴ·들ㄴㄴㄴㄴㄴ·'ㄴㄴ··ㄴㄴ···ㄴ··ㄴㄴㄴㄴㄴ,··ㄴ·ㄴ·ㄴ·ㄴㄴㄴㄴ'ㄴㄴㄴ들들ㄴ
의롭롭롭전사롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭전사롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭
의롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭
의롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭
의롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭
의롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭롭의전사전사전사전사롭롭롭롭롭아의롭롭롭롭롭의의롭롭롭롭롭롭롭롭롭롭롭롭
의롭롭전사전사전사'의전사전사군롭의롭롭롭의롭전사군군의전사전사전사전사전사롭롭롭위하아의의롭루한스크인민공화국위하의의롭롭롭롭위하위하을의롭롭롭롭
의롭전사전사전사전사'의전사전사군의의''의의의전사군군의전사전사전사전사전사롭롭롭위하아의의롭루한스크인민공화국위하의의의롭롭롭위하위하을아의롭롭롭
의롭전사전사전사전사'의루한스크'군의의''의의루한스크우리군군의전사전사전사전사전사롭롭롭위하아,의평화루한스크인민공화국위하의의평화롭롭롭위하위하을위하아롭아과
의군전사전사전사전사아의루한스크'군의의''군의루한스크우리군군의돈바스전사전사전사과롭롭롭위하아,평화평화루한스크인민공화국군의의평화롭롭롭롭위하을아아아아과
'전사전사전사전사전사아의루한스크'특별군사의''군의평화우리군군이돈바스전사전사전사과롭롭롭위하아,,·루한스크인민공화국군의의평화롭롭롭롭삶삶아아아아을
전사전사전사전사전사전사아아서을'특별군사의''군의평화우리군군이돈바스전사전사들과롭롭롭위하아,,·루한스크인민공화국국민의의평화롭롭롭ㄴ삶을아아아아을
전사전사전사전사전사전사아아서을'특별군사의''군의평화우리군군이돈바스전사전사들과롭롭롭위하아,도네츠크·루한스크인민공화국국민의의평화롭롭롭위하삶을위하아아아,
전사전사전사전사전사전사아아서작전'특별군사작전''군의평화우리군군이돈바스전사전사들과롭롭롭위하아,도네츠크·루한스크인민공화